# Solution Sklearn2 // Beau

### === Task 1 ===

1. Load the sklearn breast cancer dataset.  Perform SVC with cross-validation of leaveOneOut.

In [1]:
# Load dataset
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y = True)
print(X.shape)
print(y.shape)

# SVM model
from sklearn import svm
model = svm.SVC(kernel='linear').fit(X, y)

# CV
from sklearn.model_selection import LeaveOneOut, cross_val_score
scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

(569, 30)
(569,)


KeyboardInterrupt: 

In [ ]:
scores.mean()

### === Task 2 ===

1. Load the sklearn boston dataset.  Extract the 'RAD' feature as your single feature for your X with the normal y.   Using a validation curve to test different polynomials and identify the optimal polynomial order for the regression.  

In [ ]:
# Load dataset
from sklearn.datasets import load_boston
import pandas as pd
boston = load_boston()
boston.keys()
boston.feature_names

In [ ]:
X = boston.data[:,8].reshape(-1,1)
y = boston.target

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt 

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

plt.scatter(Xtrain.ravel(), ytrain, color='black')
axis = plt.axis()

for degree in [1, 3, 5]:
    y_test = PolynomialRegression(degree).fit(Xtrain, ytrain).predict(Xtest)
    plt.scatter(Xtest.ravel(), y_test, label='degree={0}'.format(degree))
plt.legend(loc='best')

In [ ]:
from sklearn.model_selection import validation_curve
degree = np.arange(0, 21)

train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          param_name='polynomialfeatures__degree', param_range= degree, cv=7)

plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.xlabel('degree')
plt.ylabel('score');

### === Task 3 ===

1. Load the sklearn boston dataset.  Extract the RAD feature as your single feature for your X with the normal y.   Using a learning curve to test polynomial of 2 and 9.  Check whether you found the same phenemonon as above.  How much sample is enough for 2 and 9?

In [ ]:
from sklearn.model_selection import learning_curve

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for i, degree in enumerate([2, 9]):
    N, train_lc, val_lc = learning_curve(PolynomialRegression(degree),
                                         X, y, cv=7,
                                         train_sizes=np.linspace(0.3, 1, 25))

    ax[i].plot(N, np.mean(train_lc, 1), color='blue', label='training score')
    ax[i].plot(N, np.mean(val_lc, 1), color='red', label='validation score')
    ax[i].hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0], N[-1],
                 color='gray', linestyle='dashed')

    #ax[i].set_ylim(0, 1)
    #ax[i].set_xlim(N[0], N[-1])
    ax[i].set_xlabel('training size')
    ax[i].set_ylabel('score')
    ax[i].set_title('degree = {0}'.format(degree), size=14)
    ax[i].legend(loc='best')

### === Task 4 ===

1. Load the sklearn boston dataset.  Extract the RAD feature as your single feature for your X with the normal y.   Use the grid search to identify the best polynomial order to use for prediction.  Then perform the prediction using the <code>grid_best_estimator_</code>

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(21),}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)
grid.fit(Xtrain, ytrain)
grid.best_params_

In [ ]:
Xtrain.shape

In [ ]:
model = grid.best_estimator_

y_model = model.fit(Xtrain, ytrain).predict(Xtest)

# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
MSE = mean_squared_error(ytest, y_model)
r2 = r2_score(ytest, y_model)
print("MSE: ", MSE)
print("r2: ", r2)